In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

# MBTI 별 posting 모으기

In [2]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

types

    #input_file = r'.\data\training'
    #allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
    #for file in allFile_list:
    #    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
    #    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

,posts
type,
ENFJ,190
ENFP,675
ENTJ,231
ENTP,685
ESFJ,42
ESFP,48
ESTJ,39
ESTP,89
INFJ,1470


# Resampling (Undersampling and Oversampling)

# Tokenize and Stemming
morph analysis 기법.

In [3]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
mbti= dataCombine
X_data = mbti['posts']
y_data = mbti['type']

print("normalize 시작")
normalized_text = []
for sentence in X_data.tolist():
    clean_sentence = re.sub('[^A-Za-z\s]+', '',sentence.lower())
    normalized_text.append(clean_sentence)
mbti.posts = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in mbti['posts']:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
'''tf = TfidfVectorizer().fit(stemData)
print(tf.transform(stemData).toarray())
print(tf.vocabulary_)
#print(vector.vocabulary_)'''


stemming 시작
stemming 완료


MemoryError: Unable to allocate 19.1 GiB for an array with shape (8675, 295647) and data type float64

# Model building

In [18]:
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)
preds = classifier.predict(X_test)



"X_data = np.array(X_token).reshape(-1,1)\ny_data = np.array(mbti['type']).reshape(-1,1)\n\nX_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size= 0.3, random_state=1234)\nmodel = XGBClassifier()\nmodel.fit(X_train, y_train)\npreds = classifier.predict(X_test)\n\nplot_importance(model)\npyplot.show()"